## Focus on the 1-octanethiol ligand because it has the highest H2 produced at 0.1 mM Ni Concentration

In [12]:
from rdkit import Chem
from rdkit.Chem import AllChem
from ase import Atoms
from ase.io import read, write
from ase.constraints import FixAtoms
import numpy as np
from io import StringIO
from rdkit.Chem import rdmolfiles

# --- Helper functions ---

def rdkit_to_ase(mol):
    """Convert RDKit Mol to ASE Atoms via SDF string buffer"""
    buf = StringIO()
    w = rdmolfiles.SDWriter(buf)
    w.write(mol)
    w.close()
    buf.seek(0)
    return read(buf, format="sdf")

def align_bond_to_axis(atoms, idx1, idx2, axis=(0, 0, 1)):
    """
    Rotate the molecule so that the vector from idx1 to idx2 aligns with the given axis.
    atoms: ASE Atoms object
    idx1: atom index (anchor)
    idx2: atom index (bonded atom)
    axis: target axis to align with (default z-axis)
    """
    vec = atoms[idx2].position - atoms[idx1].position
    vec /= np.linalg.norm(vec)
    target = np.array(axis) / np.linalg.norm(axis)

    v = np.cross(vec, target)
    c = np.dot(vec, target)
    if np.linalg.norm(v) < 1e-6:
        return  # already aligned
    v /= np.linalg.norm(v)
    K = np.array([[0, -v[2], v[1]],
                  [v[2], 0, -v[0]],
                  [-v[1], v[0], 0]])
    R = np.eye(3) + K + K @ K * (1 / (1 + c))

    atoms.positions = np.dot(atoms.positions - atoms[idx1].position, R.T) + atoms[idx1].position

# --- Step 0: Build thiol from SMILES ---
smiles = "CCCCCCCCS"   # 1-octanethiol
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# --- Step 1: Generate 3D geometry (deterministic) ---
params = AllChem.ETKDG()
params.randomSeed = 42
AllChem.EmbedMolecule(mol, params)
AllChem.UFFOptimizeMolecule(mol)

# --- Step 2: Convert to ASE ---
thiol = rdkit_to_ase(mol)

# --- Step 3: Identify S and C atoms ---
s_idx = [i for i, atom in enumerate(thiol) if atom.symbol == "S"][0]
s_pos_orig = thiol[s_idx].position

# Remove S–H proton to make thiolate
h_neighbors = [i for i, atom in enumerate(thiol) 
               if atom.symbol == "H" and np.linalg.norm(atom.position - s_pos_orig) < 1.5]
if h_neighbors:
    del thiol[h_neighbors[0]]

# Find the carbon attached to S
c_idx = [i for i, atom in enumerate(thiol) 
         if atom.symbol == "C" and np.linalg.norm(atom.position - s_pos_orig) < 1.9][0]

# --- Step 4: Align S–C bond along z-axis ---
align_bond_to_axis(thiol, s_idx, c_idx, axis=(0, 0, 1))

# --- Step 5: Fix sulfur atom ---
c = FixAtoms(indices=[s_idx])
thiol.set_constraint(c)

# --- Optional: save fixed thiol geometry ---
write("thiol_fixed.xyz", thiol)

# --- Step 6: Place Ni atom ---
ni = Atoms("Ni", positions=[[0, 0, 0]])

# Desired radial vectors for trigonal planar around Ni
angles = [0, 120, 240]
radius = 2.3  # Ni–S bond distance
radial_vecs = [np.array([np.cos(np.radians(a)), np.sin(np.radians(a)), 0]) for a in angles]

ligands = []
for rv in radial_vecs:
    lig = thiol.copy()
    
    # Get sulfur and SC vector for this copy
    s_pos = lig[s_idx].position
    c_pos = lig[c_idx].position
    lig_vec = c_pos - s_pos
    lig_vec /= np.linalg.norm(lig_vec)

    # Compute rotation to align lig_vec with target rv
    v = np.cross(lig_vec, rv)
    c = np.dot(lig_vec, rv)
    if np.linalg.norm(v) > 1e-6:
        v /= np.linalg.norm(v)
        K = np.array([[0, -v[2], v[1]],
                      [v[2], 0, -v[0]],
                      [-v[1], v[0], 0]])
        R = np.eye(3) + K + K @ K * (1 / (1 + c))
        lig.positions = np.dot((lig.positions - s_pos), R.T) + s_pos

    # Translate sulfur to Ni–S position
    s_pos = lig[s_idx].position
    shift = rv * radius - s_pos
    lig.translate(shift)

    ligands.append(lig)

# --- Step 7: Combine full complex ---
complex_atoms = ni
for lig in ligands:
    complex_atoms += lig

# --- Optional: save Ni-thiolate complex ---
write("ni_thiolate_complex.xyz", complex_atoms)


In [13]:
from ase.visualize import view
view(complex_atoms, viewer='x3d')

In [14]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 0: Load or build your Ni-thiolate complex ---
# complex_atoms = ... (from your previous deterministic Ni–thiolate code)

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cpu")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc

# --- Step 2: Relax the clean Ni-thiolate complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
write("ni_thiolate_relaxed.xyz", complex_atoms)
E_complex = complex_atoms.get_potential_energy()

# --- Step 3: Energy of isolated H atom ---
h_atom = Atoms("H", positions=[[0, 0, 0]])
h_atom.calc = calc
E_H = h_atom.get_potential_energy()

# --- Step 4: Add H above Ni plane ---
ads_complex = complex_atoms.copy()
# Identify Ni and S atoms
ni_pos = ads_complex[0].position
s_pos = np.array([ads_complex[i].position for i, a in enumerate(ads_complex) if a.symbol == "S"])

# Compute Ni-S plane normal
v1 = s_pos[1] - s_pos[0]
v2 = s_pos[2] - s_pos[0]
normal = np.cross(v1, v2)
normal /= np.linalg.norm(normal)

# Place H along normal, e.g., 1.6 Å above Ni
h_pos = ni_pos + normal * 1.6
ads_complex += Atoms("H", positions=[h_pos])
ads_complex.calc = calc

# --- Step 5: Relax H only, fix everything else ---
h_idx = len(ads_complex) - 1  # H is last atom
# Fix all complex atoms
fixed_idx = [i for i in range(len(ads_complex)-1)]
ads_complex.set_constraint(FixAtoms(indices=fixed_idx))
# Optional: also constrain H x and y if needed
# ads_complex.set_constraint(FixCartesian(h_idx, [0,1]))

opt_ads = FIRE(ads_complex)
opt_ads.run(fmax=0.1, steps=200)
write("ni_thiolate_H_ads.xyz", ads_complex)
E_ads_complex = ads_complex.get_potential_energy()

# --- Step 6: Compute adsorption energy ---
E_ads = E_ads_complex - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")


      Step     Time          Energy          fmax
FIRE:    0 13:56:54   -99235.501512        4.693648
FIRE:    1 13:56:55   -99239.696204        4.431303
FIRE:    2 13:56:57   -99243.288402        4.039597
FIRE:    3 13:56:58   -99246.271822        3.519145
FIRE:    4 13:57:00   -99248.662633        2.805946
FIRE:    5 13:57:01   -99250.501154        2.584249
FIRE:    6 13:57:03   -99251.854736        2.760701
FIRE:    7 13:57:04   -99252.821102        3.146559
FIRE:    8 13:57:05   -99253.503847        2.761517
FIRE:    9 13:57:07   -99253.912080        1.589362
FIRE:   10 13:57:08   -99254.039075        2.253882
FIRE:   11 13:57:10   -99254.378952        1.693074
FIRE:   12 13:57:12   -99254.872651        1.252777
FIRE:   13 13:57:13   -99255.341619        1.450156
FIRE:   14 13:57:14   -99255.736342        1.651141
FIRE:   15 13:57:16   -99256.050110        1.436502
FIRE:   16 13:57:17   -99256.261328        0.963987
FIRE:   17 13:57:19   -99256.412744        1.060780
FIRE:   18 13:

      Step     Time          Energy          fmax
FIRE:    0 13:58:49   -99279.025338        2.247015
FIRE:    1 13:58:50   -99279.074761        2.149374
FIRE:    2 13:58:52   -99279.164214        1.908456
FIRE:    3 13:58:53   -99279.269979        1.410650
FIRE:    4 13:58:55   -99279.344211        0.436731
FIRE:    5 13:58:56   -99279.318080        1.203729
FIRE:    6 13:58:58   -99279.321590        1.130886
FIRE:    7 13:58:59   -99279.327783        0.992684
FIRE:    8 13:59:00   -99279.335250        0.802652
FIRE:    9 13:59:02   -99279.342362        0.578079
FIRE:   10 13:59:03   -99279.347713        0.338250
FIRE:   11 13:59:05   -99279.350411        0.113769
FIRE:   12 13:59:06   -99279.350216        0.158799
FIRE:   13 13:59:07   -99279.350235        0.157421
FIRE:   14 13:59:09   -99279.350264        0.154681
FIRE:   15 13:59:11   -99279.350309        0.150633
FIRE:   16 13:59:12   -99279.350367        0.145351
FIRE:   17 13:59:14   -99279.350435        0.138935
FIRE:   18 13:

In [16]:
from ase.visualize import view
view(ads_complex, viewer='x3d')

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolfiles
from ase import Atoms
from ase.io import read, write
from ase.constraints import FixAtoms
import numpy as np
from io import StringIO

# --- Helper functions ---
def rdkit_to_ase(mol):
    """Convert RDKit Mol to ASE Atoms via SDF string buffer"""
    buf = StringIO()
    w = rdmolfiles.SDWriter(buf)
    w.write(mol)
    w.close()
    buf.seek(0)
    return read(buf, format="sdf")

def align_bond_to_axis(atoms, idx1, idx2, axis=(0, 0, 1)):
    """Rotate the molecule so that vector from idx1 to idx2 aligns with axis"""
    vec = atoms[idx2].position - atoms[idx1].position
    vec /= np.linalg.norm(vec)
    target = np.array(axis) / np.linalg.norm(axis)
    v = np.cross(vec, target)
    c = np.dot(vec, target)
    if np.linalg.norm(v) < 1e-6:
        return
    v /= np.linalg.norm(v)
    K = np.array([[0, -v[2], v[1]],
                  [v[2], 0, -v[0]],
                  [-v[1], v[0], 0]])
    R = np.eye(3) + K + K @ K * (1 / (1 + c))
    atoms.positions = np.dot(atoms.positions - atoms[idx1].position, R.T) + atoms[idx1].position

# --- Step 0: Build benzimidazole from SMILES ---
smiles = "[nH]1cnc2ccccc12"   # benzimidazole
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# Generate 3D geometry
params = AllChem.ETKDG()
params.randomSeed = 42
AllChem.EmbedMolecule(mol, params)
AllChem.UFFOptimizeMolecule(mol)

# Convert to ASE
ligand = rdkit_to_ase(mol)

# Pick a coordinating N atom
n_idx = [i for i, atom in enumerate(ligand) if atom.symbol == "N"][0]
c_idx = [i for i, atom in enumerate(ligand) if atom.symbol == "C"][0]

# Align N–C vector along z-axis
align_bond_to_axis(ligand, n_idx, c_idx, axis=(0, 0, 1))

# Fix N atom
ligand.set_constraint(FixAtoms(indices=[n_idx]))

# Place Ni atom
ni = Atoms("Ni", positions=[[0, 0, 0]])

# Build trigonal planar complex
angles = [0, 120, 240]
radius = 2.0  # Ni–N bond distance
radial_vecs = [np.array([np.cos(np.radians(a)), np.sin(np.radians(a)), 0]) for a in angles]

ligands = []
for rv in radial_vecs:
    lig = ligand.copy()
    n_pos = lig[n_idx].position
    c_pos = lig[c_idx].position
    lig_vec = c_pos - n_pos
    lig_vec /= np.linalg.norm(lig_vec)

    v = np.cross(lig_vec, rv)
    c = np.dot(lig_vec, rv)
    if np.linalg.norm(v) > 1e-6:
        v /= np.linalg.norm(v)
        K = np.array([[0, -v[2], v[1]],
                      [v[2], 0, -v[0]],
                      [-v[1], v[0], 0]])
        R = np.eye(3) + K + K @ K * (1 / (1 + c))
        lig.positions = np.dot((lig.positions - n_pos), R.T) + n_pos

    shift = rv * radius - n_pos
    lig.translate(shift)
    ligands.append(lig)

complex_atoms = ni
for lig in ligands:
    complex_atoms += lig

write("ni_benzimidazole_complex.xyz", complex_atoms)
print("Ni–benzimidazole complex written to ni_benzimidazole_complex.xyz")

Ni–benzimidazole complex written to ni_benzimidazole_complex.xyz


In [4]:
from ase.visualize import view
view(ligand, viewer='x3d')

In [3]:
from ase.visualize import view
view(complex_atoms, viewer='x3d')

In [5]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 0: Load your Ni–benzimidazole complex from file ---
# (generated from your RDKit → ASE workflow)
from ase.io import read
complex_atoms = read("ni_benzimidazole_complex.xyz")

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cpu")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc

# --- Step 2: Relax the clean Ni–benzimidazole complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
write("ni_benzimidazole_relaxed.xyz", complex_atoms)
E_complex = complex_atoms.get_potential_energy()

# --- Step 3: Energy of isolated H atom ---
h_atom = Atoms("H", positions=[[0, 0, 0]])
h_atom.calc = calc
E_H = h_atom.get_potential_energy()

# --- Step 4: Add H above Ni atom ---
ads_complex = complex_atoms.copy()
# Assume Ni is the first atom (index 0 in your builder code)
ni_pos = ads_complex[0].position

# Place H along +z relative to Ni, about 1.6 Å away
h_pos = ni_pos + np.array([0, 0, 1.6])
ads_complex += Atoms("H", positions=[h_pos])
ads_complex.calc = calc

# --- Step 5: Relax H only, fix the rest of the complex ---
h_idx = len(ads_complex) - 1  # last atom is H
fixed_idx = [i for i in range(len(ads_complex) - 1)]
ads_complex.set_constraint(FixAtoms(indices=fixed_idx))

opt_ads = FIRE(ads_complex)
opt_ads.run(fmax=0.1, steps=200)
write("ni_benzimidazole_H_ads.xyz", ads_complex)
E_ads_complex = ads_complex.get_potential_energy()

# --- Step 6: Compute adsorption energy ---
E_ads = E_ads_complex - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")


W0923 23:49:02.024000 43049 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


      Step     Time          Energy          fmax
FIRE:    0 23:49:07   -71960.901049       82.824448
FIRE:    1 23:49:07   -71986.843943       54.548817
FIRE:    2 23:49:09   -72000.650118       20.612986
FIRE:    3 23:49:10   -72007.903082       13.020046
FIRE:    4 23:49:11   -72012.562663       10.326780
FIRE:    5 23:49:13   -72016.112757       11.017342
FIRE:    6 23:49:14   -72019.040314        9.652960
FIRE:    7 23:49:15   -72021.521542        6.974123
FIRE:    8 23:49:16   -72023.572474        5.282444
FIRE:    9 23:49:17   -72025.342099        5.544327
FIRE:   10 23:49:18   -72027.000985        5.806211
FIRE:   11 23:49:20   -72028.646332        5.534890
FIRE:   12 23:49:21   -72030.328589        5.582175
FIRE:   13 23:49:22   -72032.016522        5.310679
FIRE:   14 23:49:23   -72033.616713        4.134395
FIRE:   15 23:49:24   -72035.031627        4.344827
FIRE:   16 23:49:26   -72036.211317        3.700750
FIRE:   17 23:49:27   -72037.167988        2.953125
FIRE:   18 23:

      Step     Time          Energy          fmax
FIRE:    0 23:52:30   -72062.292542        8.803663
FIRE:    1 23:52:31   -72062.991769        7.363756
FIRE:    2 23:52:33   -72064.060416        6.011773
FIRE:    3 23:52:34   -72065.137909        4.742077
FIRE:    4 23:52:35   -72065.952380        3.710949
FIRE:    5 23:52:36   -72066.553228        2.770421
FIRE:    6 23:52:37   -72066.961556        1.615119
FIRE:    7 23:52:38   -72067.145415        1.033985
FIRE:    8 23:52:39   -72067.141423        1.400324
FIRE:    9 23:52:41   -72067.147311        1.378829
FIRE:   10 23:52:42   -72067.158718        1.335122
FIRE:   11 23:52:43   -72067.174916        1.268077
FIRE:   12 23:52:45   -72067.194799        1.176950
FIRE:   13 23:52:47   -72067.216929        1.063110
FIRE:   14 23:52:49   -72067.239624        0.933511
FIRE:   15 23:52:51   -72067.261192        0.807017
FIRE:   16 23:52:53   -72067.282158        0.714379
FIRE:   17 23:52:54   -72067.301337        0.721382
FIRE:   18 23:

In [6]:
from ase.visualize import view
view(ads_complex, viewer='x3d')